<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [2]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2()

Run date time (IST): 2024-12-24 23:13:38
-------------------
qualified stocks: 163
with latest results: 152
still star stocks: 88


In [3]:
# x24
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
6,AMBUJACEM,543.10,733.0,-11.26,43.67,86.50,-1.37,34.97,27.97,12.18,XY24,BTT,0,0,H,CEMENT,0.0
3,ADANIPORTS,1182.20,1583.0,-14.33,-5.49,61.35,-0.81,33.90,34.51,16.75,XY24,NTT,1,1,M,MISC,NaN
26,BOROLTD,396.10,500.5,0.79,NaN,15.95,-1.58,26.36,26.36,31.90,XY24,ATH,1,0,M,GLASS,0.0
127,ROSSARI,784.95,965.0,-2.00,NaN,36.20,0.47,22.94,99.84,17.11,XY24,NTT,1,1,M,CHEMICALS,-1.0
23,BIOCON,344.50,397.0,4.39,-41.24,68.10,4.54,15.24,38.34,40.38,XY24,NTT,1,1,H,PHARMA,0.0
152,UTIAMC,1230.15,1395.0,13.80,-6.77,25.15,-0.06,13.40,13.40,55.28,XY24,ATH,1,1,M,FINANCE,0.0
88,JSWHL,14181.40,7487.0,63.64,109.06,3.07,0.24,-47.21,36.94,183.89,XY24,BTT,1,0,H,FINANCE,NaN


In [4]:
# watchlist
df_tmp = df_prospects_features[df_prospects_features['InFolio'] == -1].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
110,NIACL,198.01,354.90,-14.28,7.72,90.80,-2.10,79.23,79.23,16.09,XR,ATH,1,0,M,INSURANCE,-1.0
0,3MINDIA,30993.90,50570.00,-7.69,-23.69,38.65,1.04,63.16,29.12,9.76,X40N,BTT,1,1,M,MISC,-1.0
147,TITAN,3353.40,4713.00,-3.22,2.20,61.96,-1.28,40.54,14.93,7.45,X40,BTT,1,1,H,JEWELLERY,-1.0
29,BRITANNIA,4744.10,6446.05,-11.62,-3.11,35.58,0.84,35.88,35.88,2.95,AR,ATH,1,1,L,FMCG,-1.0
108,NESTLEIND,2166.70,2925.00,-12.10,-20.92,28.83,0.70,35.00,27.17,0.70,X40,BTT,0,0,H,FMCG,-1.0
119,PGHH,14754.80,19839.00,-8.94,-8.40,14.11,-0.91,34.46,23.69,0.00,X40,BTT,0,0,H,FMCG,-1.0
92,LICI,890.65,1188.10,-10.22,-11.11,47.24,-1.56,33.40,33.40,15.06,X200,ATH,1,0,M,INSURANCE,-1.0
13,ASTRAZEN,6423.20,8410.00,0.14,88.24,52.15,-0.53,30.93,24.08,30.78,X40N,BTT,1,0,M,PHARMA,-1.0
127,ROSSARI,784.95,965.00,-2.00,NaN,36.20,0.47,22.94,99.84,17.11,XY24,NTT,1,1,M,CHEMICALS,-1.0
107,NEOGEN,2003.85,2434.00,18.01,NaN,76.07,-1.07,21.47,17.59,67.98,SR,BTT,1,0,L,CHEMICALS,-1.0


In [5]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
18,BAJAJ-AUTO,8778.05,12666.40,-9.51,64.37,16.56,0.11,44.30,44.30,36.89,X40,ATH,1,1,H,AUTO,NaN
147,TITAN,3353.40,4713.00,-3.22,2.20,61.96,-1.28,40.54,14.93,7.45,X40,BTT,1,1,H,JEWELLERY,-1.0
59,GLAXO,2264.60,3182.00,-7.53,0.58,34.97,-0.80,40.51,34.57,30.37,X40,BTT,1,0,H,PHARMA,NaN
108,NESTLEIND,2166.70,2925.00,-12.10,-20.92,28.83,0.70,35.00,27.17,0.70,X40,BTT,0,0,H,FMCG,-1.0
119,PGHH,14754.80,19839.00,-8.94,-8.40,14.11,-0.91,34.46,23.69,0.00,X40,BTT,0,0,H,FMCG,-1.0
118,PFIZER,4803.85,6318.30,-4.61,-5.72,7.36,-0.59,31.53,31.53,17.64,X40,ATH,1,0,H,PHARMA,NaN
5,AKZOINDIA,3484.40,4433.10,12.95,7.92,26.99,-0.56,27.23,27.23,56.73,X40,ATH,1,1,H,PAINTS,NaN
64,HAVELLS,1674.15,2082.40,-5.04,6.12,44.17,0.80,24.39,24.39,30.47,X40,ATH,1,1,H,ELECTRICAL,NaN
128,SANOFI,6135.35,7552.00,-15.16,46.19,49.08,-0.77,23.09,64.17,1.34,X40,NTT,0,0,H,PHARMA,NaN
17,AXISBANK,1078.90,1317.30,-7.15,-24.78,18.40,-0.02,22.10,22.10,5.44,X40,ATH,1,1,H,BANKS,NaN


In [6]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
50,EQUITASBNK,65.16,113.10,-22.43,NaN,74.85,-0.14,73.57,73.57,4.66,X40N,ATH,1,0,M,BANKS,NaN
101,MOTILALOFS,918.95,1537.38,33.96,38.70,60.74,-0.94,67.30,67.30,198.86,X40N,ATH,1,1,M,FINANCE,NaN
0,3MINDIA,30993.90,50570.00,-7.69,-23.69,38.65,1.04,63.16,29.12,9.76,X40N,BTT,1,1,M,MISC,-1.0
139,SYMPHONY,1318.80,2016.84,3.44,NaN,43.56,2.68,52.93,52.93,61.68,X40N,ATH,1,1,M,DURABLES,0.0
71,HONAUT,41005.95,58518.52,-14.28,-15.36,50.31,0.88,42.71,42.71,16.30,X40N,ATH,1,1,M,ELECTRICAL,0.0
155,VINATIORGA,1693.85,2325.26,-7.69,-8.91,3.68,2.08,37.28,37.28,15.69,X40N,ATH,1,0,M,CHEMICALS,NaN
13,ASTRAZEN,6423.20,8410.00,0.14,88.24,52.15,-0.53,30.93,24.08,30.78,X40N,BTT,1,0,M,PHARMA,-1.0
7,ANGELONE,2900.40,3791.17,9.16,-2.69,72.39,1.43,30.71,30.71,41.39,X40N,ATH,1,1,M,FINANCE,NaN
54,FINCABLES,1270.10,1657.02,-1.01,76.01,92.02,10.25,30.46,30.46,52.28,X40N,ATH,1,1,M,CABLES,NaN
51,ERIS,1342.25,1522.70,17.74,114.14,6.13,0.42,13.44,13.44,61.46,X40N,ATH,1,1,M,PHARMA,NaN


In [7]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
19,BAJAJHIND,31.07,442.93,-13.69,NaN,54.60,-0.10,1325.59,1325.59,24.03,XR,ATH,1,0,M,SUGAR,0.0
8,APEX,241.43,849.16,1.87,NaN,74.23,-0.26,251.72,251.72,26.97,AR,ATH,1,0,L,MISC,0.0
35,CENTRALBK,54.23,190.06,-9.62,-12.99,26.38,-0.02,250.47,250.47,11.24,XR,ATH,1,0,M,BANKS,NaN
121,PNB,101.64,232.89,-12.38,-51.70,42.33,0.26,129.13,129.13,16.20,XR,ATH,1,1,M,BANKS,0.0
140,TASTYBITE,10053.00,20387.66,-13.57,NaN,11.04,-0.37,102.80,102.80,7.57,XR,ATH,1,0,L,FMCG,0.0
159,WIPRO,305.30,611.00,17.42,30.77,79.14,-0.84,100.13,15.02,39.74,XR,NTT,1,1,M,IT,0.0
80,INFIBEAM,25.82,49.37,-16.71,NaN,55.21,-0.50,91.21,91.21,20.26,AR,ATH,1,1,L,IT,NaN
41,DCBBANK,122.50,232.98,-2.00,NaN,77.30,-0.40,90.19,90.19,11.20,AR,ATH,1,1,L,BANKS,NaN
116,PARAGMILK,184.76,348.29,-6.69,NaN,7.98,-1.73,88.51,88.51,17.54,AR,ATH,1,0,L,FMCG,NaN
146,VEEDOL,1682.35,3090.13,-15.12,NaN,31.90,-0.17,83.68,83.68,28.55,XR,ATH,1,1,M,CHEMICALS,NaN


In [8]:
df_prospects_features.columns

Index(['Symbol', 'Close', 'FTT', 'Dev%_200', 'Dev%_PE', 'RSP', 'Today%',
       'FTT%', 'ATH%', 'Gained%', 'Criteria', 'Strategy', 'LatestQtr',
       'StarStock', 'Conviction', 'Category', 'InFolio'],
      dtype='object')